# 1. Setup

In [ ]:
!pip install transformers==4.6.1
!pip install SentencePiece==0.1.95
!pip install datasets

In [ ]:
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

2021-07-02 09:24:58.901948: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Downloading: 100% 629/629 [00:00<00:00, 784kB/s]
Downloading: 100% 268M/268M [00:04<00:00, 62.3MB/s]
Downloading: 100% 232k/232k [00:00<00:00, 1.72MB/s]
Downloading: 100% 48.0/48.0 [00:00<00:00, 49.4kB/s]
[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [ ]:
!pwd

/content


In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("eduge")

Couldn't find file locally at eduge/eduge.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.8.0/datasets/eduge/eduge.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/eduge/eduge.py.
Using custom data configuration default
Reusing dataset eduge (/root/.cache/huggingface/datasets/eduge/default/0.0.0/533d404905b3b3371f97e26f3dd3e73f45582dfb9ce96b9afd42bbf7fb1a87f8)


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'news'],
        num_rows: 60528
    })
    test: Dataset({
        features: ['label', 'news'],
        num_rows: 15133
    })
})

In [ ]:
dataset['train']['label'][0], dataset['train']['news'][0]

(5,
 'Манай сонин зун, өвлийн олимпийн наадам, хөлбөмбөгийн дэлхий, Европын аварга шалгаруулах тэмцээн, Азийн зуны наадам зэрэг спортын томоохон тэмцээн уралдааны өмнө хуваарийг нь уншигч та бүхэндээ өргөн барьдаг уламжлалтай билээ. Энэ удаад “Сочи-2014” өвлийн олимпийн XXII наадмын хуваарийг Улаанбаатарын цагт хөрвүүлэн хүргэж байна. Уншигч таныг энэхүү хөтөлбөртэй олимпийн наадам өндөрлөтөл хамт байна гэдэгт итгэлтэй байна. II.06-07 14:00 Сноуборд. Эр. Slopestyle. Урьдчилсан үзүүлбэр.\xa0 18:00 Сноуборд. Эм. Slopestyle. Урьдчилсан үзүүлбэр.\xa0 22:00 Фрийстайл. Эм. Могул. Урьдчилсан үзүүлбэр 1.\xa0 23:30 Уран гулгалт. Баг. Эрэгтэйн богино үзүүлбэр.\xa0 01:10 Уран гулгалт. Баг. Хосын богино үзүүлбэр.\xa0 II.07-08 00:14 Олимпийн наадмын нээлтийн ажиллагаа. II.08-09 13:30 Сноуборд. Эр. Slopestyle. Хагас шигшээ үзүүлбэр. 16:00 Хоккей. Эм. “А” хэсэг. АНУ - Финланд. 16:45 Сноуборд. Эр. Slopestyle. Шигшээ үзүүлбэр. 18:00 Гүйлтийн цана. Эм. Скиатлон. 7.5км сонгомол тэшилт + 7.5км чөлөөт тэши

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-base-mongolian-uncased', use_fast=True)

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='tugstugi/bert-base-mongolian-uncased', vocab_size=32000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
tokenizer("Сайн байна уу?", "Энэ бол хоёр дахь өгүүлбэр")

{'input_ids': [4, 107, 11, 96, 45, 5, 16, 28, 49, 150, 7417, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples['news'], truncation=True, max_length=128)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'news', 'token_type_ids'],
        num_rows: 60528
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'news', 'token_type_ids'],
        num_rows: 15133
    })
})

In [ ]:
# encoded_dataset['train']['input_ids'][0]

In [ ]:
dataset['train'][0]['news']

'Манай сонин зун, өвлийн олимпийн наадам, хөлбөмбөгийн дэлхий, Европын аварга шалгаруулах тэмцээн, Азийн зуны наадам зэрэг спортын томоохон тэмцээн уралдааны өмнө хуваарийг нь уншигч та бүхэндээ өргөн барьдаг уламжлалтай билээ. Энэ удаад “Сочи-2014” өвлийн олимпийн XXII наадмын хуваарийг Улаанбаатарын цагт хөрвүүлэн хүргэж байна. Уншигч таныг энэхүү хөтөлбөртэй олимпийн наадам өндөрлөтөл хамт байна гэдэгт итгэлтэй байна. II.06-07 14:00 Сноуборд. Эр. Slopestyle. Урьдчилсан үзүүлбэр.\xa0 18:00 Сноуборд. Эм. Slopestyle. Урьдчилсан үзүүлбэр.\xa0 22:00 Фрийстайл. Эм. Могул. Урьдчилсан үзүүлбэр 1.\xa0 23:30 Уран гулгалт. Баг. Эрэгтэйн богино үзүүлбэр.\xa0 01:10 Уран гулгалт. Баг. Хосын богино үзүүлбэр.\xa0 II.07-08 00:14 Олимпийн наадмын нээлтийн ажиллагаа. II.08-09 13:30 Сноуборд. Эр. Slopestyle. Хагас шигшээ үзүүлбэр. 16:00 Хоккей. Эм. “А” хэсэг. АНУ - Финланд. 16:45 Сноуборд. Эр. Slopestyle. Шигшээ үзүүлбэр. 18:00 Гүйлтийн цана. Эм. Скиатлон. 7.5км сонгомол тэшилт + 7.5км чөлөөт тэшилтийн

In [ ]:
mlm = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-base-mongolian-cased')

In [ ]:
## declare task ##
pipe = pipeline(task="fill-mask", model=mlm, tokenizer=tokenizer)

## example ##
input_  = 'Монгол бол [MASK] онцлогтой улс'

output_ = pipe(input_)
for i in range(len(output_)):
    print(output_[i])

{'sequence': 'монгол бол  таараа онцлогтой улс', 'score': 0.5089596509933472, 'token': 14462, 'token_str': 'таараа'}
{'sequence': 'монгол бол  мэдрэлийн онцлогтой улс', 'score': 0.14659366011619568, 'token': 6011, 'token_str': 'мэдрэлийн'}
{'sequence': 'монгол бол  үеэ онцлогтой улс', 'score': 0.037332162261009216, 'token': 6203, 'token_str': 'үеэ'}
{'sequence': 'монгол бол  мүхаүт онцлогтой улс', 'score': 0.03441431000828743, 'token': 9468, 'token_str': 'мүхаүт'}
{'sequence': 'монгол бол тай онцлогтой улс', 'score': 0.031810462474823, 'token': 55, 'token_str': 'тай'}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('tugstugi/bert-base-mongolian-uncased', num_labels=9)

Some weights of the model checkpoint at tugstugi/bert-base-mongolian-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tugstugi

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
metric_name = "accuracy"
batch_size = 16

args = TrainingArguments(
    "eduge",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
metric = load_metric('accuracy')

In [ ]:
import numpy as np
task = "eduge"
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'news', 'token_type_ids'],
        num_rows: 60528
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'news', 'token_type_ids'],
        num_rows: 15133
    })
})

In [ ]:
model.resize_token_embeddings(32001)

Embedding(32001, 768)

In [ ]:
model.config

BertConfig {
  "_name_or_path": "tugstugi/bert-base-mongolian-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "AlbertTokenizer",
  "transformers_version": "4.6.1",
  "type_voca

In [ ]:
 model.bert.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(32000, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
model.config

BertConfig {
  "_name_or_path": "tugstugi/bert-base-mongolian-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "AlbertTokenizer",
  "transformers_version": "4.6.1",
  "type_voca

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi

Fri Jul  2 09:53:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    29W /  70W |   4638MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model.bert.embeddings.word_embeddings

Embedding(32001, 768)

In [ ]:
model.bert.embeddings.word_embeddings.padding_idx

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '[CLS]',
 'cls_token': '[CLS]',
 'eos_token': '[SEP]',
 'mask_token': '[MASK]',
 'pad_token': '<pad>',
 'sep_token': '[SEP]',
 'unk_token': '<unk>'}

In [ ]:
mkdir eduge-finetuned

In [ ]:
!gsutil -m cp gs://eduge_dataset/Eduge-finetuned/* eduge-finetuned/

Copying gs://eduge_dataset/Eduge-finetuned/config.json...
Copying gs://eduge_dataset/Eduge-finetuned/added_tokens.json...
Copying gs://eduge_dataset/Eduge-finetuned/scheduler.pt...
Copying gs://eduge_dataset/Eduge-finetuned/pytorch_model.bin...
Copying gs://eduge_dataset/Eduge-finetuned/special_tokens_map.json...
Copying gs://eduge_dataset/Eduge-finetuned/optimizer.pt...
Copying gs://eduge_dataset/Eduge-finetuned/rng_state.pth...
Copying gs://eduge_dataset/Eduge-finetuned/spiece.model...
Copying gs://eduge_dataset/Eduge-finetuned/tokenizer.json...
Copying gs://eduge_dataset/Eduge-finetuned/tokenizer_config.json...
Copying gs://eduge_dataset/Eduge-finetuned/trainer_state.json...
Copying gs://eduge_dataset/Eduge-finetuned/training_args.bin...


In [ ]:
from transformers import pipeline

In [ ]:
nlp = pipeline("text-classification", model='./eduge-finetuned/', config='./eduge-finetuned/')

In [ ]:
nlp('Аюулгүй байдал, гадаад бодлогын байнгын хорооны өнөөдрийн хуралдаанаар зарим улсад суух Элчин сайдуудын томилгоог хэлэлцэж байна. Засгийн газрын хуралдаанаар 25 улсад суух Элчин сайдад нэр дэвшигчдийг дэмжсэн билээ. Элчин сайдад нэр дэвшигчдийг Ерөнхийлөгчийн Тамгын газрын дарга Я.Содбаатар танилцуулав.')

[{'label': 'LABEL_4', 'score': 0.9985010623931885}]

In [ ]:
nlp('Олон улсын олимпын хороо /ОУОХ/-ны Гүйцэтгэх зөвлөл нь урьдчилан хүлээн зөвшөөрсөн Олон улсын спортын доорх зургаан холбоог Олимпын хартийн дагуу албан ёсоор хүлээн зөвшөөрөв. Олон улсын дэмжигчдийн холбоо /ICU - International Cheerleading Union, 2016 онд урьдчилан статусаар хүлээн зөвшөөрсөн/ Олон улсын Муайтай холбоо /IFU - International Federation of Muaytai Associations, 2016 онд урьдчилан статусаар хүлээн зөвшөөрсөн/ Олон улсын самбо бөхийн холбоо /FIAS - International Sambo Federation, 2018 онд урьдчилан статусаар хүлээн зөвшөөрсөн/ Олон улсын Айс-Стокийн холбоо /IFI - International Federation Icestocksport, 2018 онд урьдчилан статусаар хүлээн зөвшөөрсөн/ Кикбоксын байгууллагуудын дэлхийн холбоо /WAKO - World Association of Kickboxing Organisations, 2018 онд урьдчилан статусаар хүлээн зөвшөөрсөн/ Дэлхийн Лякросс /WL - World Lacrosse, 2018 онд урьдчилан статусаар хүлээн зөвшөөрсөн')

[{'label': 'LABEL_5', 'score': 0.999584436416626}]